In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import ShuffleSplit, train_test_split, KFold
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, precision_score, recall_score
random_num = 2018


cancer = load_breast_cancer()
features = cancer.feature_names
x_data = cancer.data
y_data = cancer.target
print(x_data.shape)

# x_data = pd.DataFrame(x_data,columns=features)
# y_data = pd.Series(y_data,index=x_data.index)

# split data into training and testing sets
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=random_num)

(569, 30)


In [2]:
def validation(model, xg_data, y_data):
    y_data_proba = model.predict(xg_data)
    y_data_pred = (y_data_proba>=0.5).astype(int)
    
    acc = accuracy_score(y_data, y_data_pred)
    prec = precision_score(y_data, y_data_pred)
    recall = recall_score(y_data, y_data_pred)
    roc = roc_auc_score(y_data, y_data_proba)
    return acc, prec, recall, roc

In [5]:
xg_train_0 = xgb.DMatrix(x_train, label=y_train)
xg_val = xgb.DMatrix(x_val, label=y_val)
n_fold = 5
num_round = 4
verbose_eval = 3
watch_list = [(xg_val, 'xg_val')]
params = {'objective': 'binary:logistic', 'verbose': False, 'eval_metric': ['auc']}
print('full train\t'); 
model_0 = xgb.train(params, xg_train_0, n_fold*num_round, watch_list, verbose_eval=verbose_eval)
print(validation(model_0, xg_val, y_val))

kf = KFold(n_fold, True, random_num)
xgboost_model = None
for i, kf_idx in enumerate(kf.split(x_train)):
    x_train_kf = x_train[kf_idx[1]]
    y_train_kf = y_train[kf_idx[1]]
    xg_train_kf = xgb.DMatrix(x_train_kf, label=y_train_kf)
    xgboost_model1 = xgb.train(params, xg_train_kf, num_round, watch_list, verbose_eval=verbose_eval, xgb_model=xgboost_model)
    params.update({
        'process_type': 'update',
        'update': 'refresh',
        'refresh_leaf': True,
    })
    print(params)
    print(validation(xgboost_model1, xg_val, y_val))
    xgboost_model = xgboost_model1
    del xgboost_model1

full train	
[0]	xg_val-auc:0.973142
[3]	xg_val-auc:0.980574
[6]	xg_val-auc:0.99223
[9]	xg_val-auc:0.993243
[12]	xg_val-auc:0.992568
[15]	xg_val-auc:0.992905
[18]	xg_val-auc:0.994932
[19]	xg_val-auc:0.994595
(0.956140350877193, 0.96, 0.972972972972973, 0.9945945945945946)
[0]	xg_val-auc:0.947128
[3]	xg_val-auc:0.980912
{'objective': 'binary:logistic', 'verbose': False, 'eval_metric': ['auc'], 'process_type': 'update', 'update': 'refresh', 'refresh_leaf': True}
(0.9035087719298246, 0.9090909090909091, 0.9459459459459459, 0.9809121621621621)


XGBoostError: b'[17:38:09] /root/xgboost/src/tree/updater_colmaker.cc:119: Check failed: tree.param.num_nodes == tree.param.num_roots (7 vs. 1) ColMaker: can only grow new tree\n\nStack trace returned 10 entries:\n[bt] (0) /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/xgboost-0.71-py3.6.egg/xgboost/libxgboost.so(_ZN4dmlc15LogMessageFatalD1Ev+0x3c) [0x7f19dd84adcc]\n[bt] (1) /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/xgboost-0.71-py3.6.egg/xgboost/libxgboost.so(_ZN7xgboost4tree8ColMakerINS0_9GradStatsENS0_12NoConstraintEE7Builder8InitDataERKSt6vectorINS_6detail20GradientPairInternalIfEESaIS9_EERKNS_7DMatrixERKNS_7RegTreeE+0xd5) [0x7f19dda0a755]\n[bt] (2) /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/xgboost-0.71-py3.6.egg/xgboost/libxgboost.so(_ZN7xgboost4tree8ColMakerINS0_9GradStatsENS0_12NoConstraintEE6UpdateEPNS_16HostDeviceVectorINS_6detail20GradientPairInternalIfEEEEPNS_7DMatrixERKSt6vectorIPNS_7RegTreeES